In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import getopt
import sys
import os
import math
import time
import argparse
import torch
import torch as T
from torch.autograd import Variable as var
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.nn.utils import clip_grad_norm_

#torch.manual_seed(1)


import os
from io import open
import inspect
import numpy as np
import pandas as pd

import torch
import torchtext
from torchtext.datasets import BABI20
from torchtext.data import Dataset, Field, Example, Iterator

import argparse
from torchtext import datasets
from torchtext.datasets.babi import BABI20Field
#from models.UTransformer import BabiUTransformer
#from models.common_layer import NoamOpt
import torch.nn as nn
from copy import deepcopy


import warnings
warnings.filterwarnings('ignore')

import numpy as np
import getopt
import sys
import os
import math
import time
import argparse
from visdom import Visdom

sys.path.insert(0, os.path.join('..', '..'))

In [3]:


import torch as T
from torch.autograd import Variable as var
import torch.nn.functional as F
import torch.optim as optim

from torch.nn.utils import clip_grad_norm_


In [4]:
with __import__('importnb').Notebook(): 
    from dnc import DNC
    from util import *

In [5]:
def arrangestory(batch_story): # (batch,length,size)
    strysum =torch.sum(batch_story,(0,2)) #1D
    fill_length=len(strysum[torch.where(strysum!=0)])
    red_story=batch_story[:,:fill_length,:]
    red_story =torch.flip(red_story, dims=[1])
    flat_story =red_story.view(len(batch_story),-1)
    return flat_story #(batch, red_length * size ) 2D

def prep_data(batch,vocab_size):
    batch_size=batch.story.size()[0]
    flat_story =arrangestory(batch.story)
    story_OH =torch.nn.functional.one_hot(flat_story,num_classes=vocab_size) #(batch, seq_len, vocab_len)
    query_OH=torch.nn.functional.one_hot(batch.query,num_classes=vocab_size) #(batch, que_len=3, vocab_len)
    answer_OH=torch.nn.functional.one_hot(batch.answer,num_classes=vocab_size) #(batch,1, vocab_len)
    querystop =torch.nn.functional.one_hot(torch.tensor([vocab_size-1]*batch_size),num_classes=vocab_size)
    querystop=querystop.view(batch_size,1,-1) #2D -> 3D
    x =torch.cat((story_OH,query_OH,querystop),1)
    y =answer_OH.view(batch_size,vocab_size)
    ans_id =batch.answer.view(-1) #1D (batch_size)



    #x = var(x.detach().clone().to(torch.float).cuda() )
    #y = var(y.detach().clone().to(torch.float).cuda() )
    x = var(x.detach().clone().to(torch.float).cuda() )
    y = var(y.detach().clone().to(torch.float).cuda() )
    ans_id=var(ans_id.cuda() )
    #ans_id=var(ans_id.detach().clone().cuda() )

    return x,y,ans_id #yはonehotなので2Dだけどans_idは1D !!

In [6]:
parser = argparse.ArgumentParser(description='PyTorch Differentiable Neural Computer')
parser.add_argument('-input_size', type=int, default=6, help='dimension of input feature')
parser.add_argument('-rnn_type', type=str, default='lstm', help='type of recurrent cells to use for the controller')
parser.add_argument('-nhid', type=int, default=64, help='number of hidden units of the inner nn')
parser.add_argument('-dropout', type=float, default=0, help='controller dropout')
parser.add_argument('-memory_type', type=str, default='dnc', help='dense or sparse memory: dnc | sdnc | sam')

parser.add_argument('-nlayer', type=int, default=1, help='number of layers')
parser.add_argument('-nhlayer', type=int, default=2, help='number of hidden layers')
parser.add_argument('-lr', type=float, default=1e-4, help='initial learning rate')
parser.add_argument('-optim', type=str, default='adam', help='learning rule, supports adam|rmsprop')
parser.add_argument('-clip', type=float, default=50, help='gradient clipping')

parser.add_argument('-batch_size', type=int, default=100, metavar='N', help='batch size')
parser.add_argument('-mem_size', type=int, default=20, help='memory dimension')
parser.add_argument('-mem_slot', type=int, default=16, help='number of memory slots')
parser.add_argument('-read_heads', type=int, default=4, help='number of read heads')
parser.add_argument('-sparse_reads', type=int, default=10, help='number of sparse reads per read head')
parser.add_argument('-temporal_reads', type=int, default=2, help='number of temporal reads')

parser.add_argument('-sequence_max_length', type=int, default=4, metavar='N', help='sequence_max_length')
parser.add_argument('-curriculum_increment', type=int, default=0, metavar='N', help='sequence_max_length incrementor per 1K iterations')
parser.add_argument('-curriculum_freq', type=int, default=1000, metavar='N', help='sequence_max_length incrementor per 1K iterations')
parser.add_argument('-cuda', type=int, default=-1, help='Cuda GPU ID, -1 for CPU')

parser.add_argument('-iterations', type=int, default=100000, metavar='N', help='total number of iteration')
parser.add_argument('-summarize_freq', type=int, default=100, metavar='N', help='summarize frequency')
parser.add_argument('-check_freq', type=int, default=100, metavar='N', help='check point frequency')
parser.add_argument('-visdom', action='store_true', help='plot memory content on visdom per -summarize_freq steps')

_StoreTrueAction(option_strings=['-visdom'], dest='visdom', nargs=0, const=True, default=False, type=None, choices=None, help='plot memory content on visdom per -summarize_freq steps', metavar=None)

In [14]:
args = parser.parse_args(args="-rnn_type lstm -nhid 256 -lr 1e-3 -optim rmsprop -batch_size 50 -mem_size 64 -mem_slot 256 -sequence_max_length 50 -cuda 0 -iterations 1000 -summarize_freq 10 -check_freq 11000".split(" "))

#viz = Visdom()
# assert viz.check_connection()

if args.cuda != -1:
  print('Using CUDA.')
  #T.manual_seed(1111)
else:
  print('Using CPU.')


Using CUDA.


In [8]:
def llprint(message):
  sys.stdout.write(message)
  sys.stdout.flush()


In [9]:
def my_criterion(predictions, targets):
  return T.mean(
      -1 * F.logsigmoid(predictions) * (targets) - T.log(1 - F.sigmoid(predictions) + 1e-9) * (1 - targets)
  )

In [10]:
def get_babidata_task_etc(task,batch_size, length, size): #~size
    """(batch_size=config.batch_size, 
                                                            root='.data', 
                                                            memory_size=70, 
                                                            task=config.task, 
                                                            joint=False,
                                                            tenK=False, 
                                                            only_supporting=False, 
                                                            sort=False, 
                                                            shuffle=True)"""
    text = BABI20Field(length)
    train, val, test = datasets.BABI20.splits(text, root='.data', task=task, joint=False,
                                            tenK=True, only_supporting=False)
    text.build_vocab(train)
    #text.vocab.append_token
    vocab_len = len(text.vocab.freqs) +1
    vocab_lenplus =vocab_len+1 # "?"
    print("VOCAB LEN PLUS:",vocab_lenplus )
    train_iter,val_iter,test_iter = Iterator.splits((train, val, test),batch_size=batch_size)
    return train_iter, val_iter, test_iter,vocab_lenplus,text

In [13]:
def main_dnc_babi(task):
  print("==========task:",task)
  dirname = os.path.dirname(os.path.abspath('__file__'))
  ckpts_dir = 'checkpoints'
  if not os.path.isdir(ckpts_dir):
    os.mkdir(ckpts_dir)

  
  sequence_max_length = args.sequence_max_length
  check_freq = args.check_freq

  # input_size = output_size = args.input_size
  mem_slot = args.mem_slot
  mem_size = args.mem_size
  read_heads = args.read_heads

  print("batch_size:",args.batch_size)
  train_iter, val_iter, test_iter, vocab_size, text_field =get_babidata_task_etc(task,args.batch_size,sequence_max_length,6)
  args.batch_size
  if args.memory_type == 'dnc':
    rnn = DNC(
        #input_size=args.input_size,
        input_size=vocab_size,
        hidden_size=args.nhid,
        rnn_type=args.rnn_type,
        num_layers=args.nlayer,
        num_hidden_layers=args.nhlayer,
        dropout=args.dropout,
        nr_cells=mem_slot,
        cell_size=mem_size,
        read_heads=read_heads,
        gpu_id=args.cuda,
        debug=args.visdom,
        batch_first=True,
        independent_linears=True)
  #batchfirst
  print(rnn)
  # register_nan_checks(rnn)

  if args.optim == 'adam':
    optimizer = optim.Adam(rnn.parameters(), lr=args.lr, eps=1e-9, betas=[0.9, 0.98]) # 0.0001
  elif args.optim == 'adamax':
    optimizer = optim.Adamax(rnn.parameters(), lr=args.lr, eps=1e-9, betas=[0.9, 0.98]) # 0.0001
  elif args.optim == 'rmsprop':
    optimizer = optim.RMSprop(rnn.parameters(), lr=args.lr, momentum=0.9, eps=1e-10) # 0.0001
  elif args.optim == 'sgd':
    optimizer = optim.SGD(rnn.parameters(), lr=args.lr) # 0.01
  elif args.optim == 'adagrad':
    optimizer = optim.Adagrad(rnn.parameters(), lr=args.lr)
  elif args.optim == 'adadelta':
    optimizer = optim.Adadelta(rnn.parameters(), lr=args.lr)

  criterion = nn.CrossEntropyLoss()
  #criterion = my_criterion
  save_dir="dnc_param"
  save_path=os.path.join(save_dir,"task"+str(task)+".pth")

  iterations = args.iterations
  summarize_freq = args.summarize_freq
  print("summarize_freq:",summarize_freq)
  last_save_losses = []
  last_val_losses =[]
  val_acc=[]
  val_bestacc=0
  val_bestloss=0
  bestepoch =0

  if args.cuda != -1:
    rnn = rnn.cuda(args.cuda)
  
  rnn.train()
  (chx, mhx, rv) = (None, None, None)
  for epoch in range(iterations + 1):
    llprint("\rIteration {ep}/{tot}".format(ep=epoch, tot=iterations))
    
    #generate_data(batch_size, random_length, args.input_size, args.cuda)
    for i,batch in enumerate(train_iter):
      llprint("\rbatch {bt}".format(bt=i))
      batch_size = batch.batch_size
      train_x,train_y,ans_id =prep_data(batch,vocab_size)
      #torch.autograd.set_detect_anomaly(True)
      if train_x.size()[0]!=args.batch_size:continue
      #if rnn.debug:
      #  output, (chx, mhx, rv), v = rnn(train_x, (None, mhx, None), reset_experience=True, pass_through_memory=True)
      #else:
      output, (chx, mhx, rv) = rnn(train_x, (None, mhx, None), reset_experience=True, pass_through_memory=True)
      
      optimizer.zero_grad()
      last_output =output[:,-1,:]
      #print("last_output.size():",last_output.size()," ans_id.size():",ans_id.size())
      #loss = criterion((last_output),train_y)
      loss = criterion(last_output,ans_id)
      if i==0:print("oneloss:",loss.item())
      loss.backward()
      T.nn.utils.clip_grad_norm_(rnn.parameters(), args.clip)
      optimizer.step()
      loss_value = loss.item()
      last_save_losses.append(loss_value)

      mhx = { k : (v.detach() if isinstance(v, var) else v) for k, v in mhx.items() }#~~??

    #T.nn.utils.clip_grad_norm_(rnn.parameters(), args.clip) ~~??

    #validation
    with torch.no_grad():
      for i,batch in enumerate(val_iter):
        #print("\rIteration {ep}/{tot}".format(ep=epoch, tot=iterations))
        batch_size=batch.batch_size
        val_x,val_y,val_ansid =prep_data(batch,vocab_size)
        if val_x.size()[0]!=args.batch_size:continue
        #print("valx.size():",val_x.size()," valy.size():",val_y.size())
        output, (chx, mhx, rv) = rnn(val_x, (None, mhx, None), reset_experience=True, pass_through_memory=True)
        last_output =output[:,-1,:]
        optimizer.zero_grad()    
        loss = criterion(last_output,val_ansid)
        #loss = criterion((last_output),val_y)
        loss_value = loss.item()
        last_val_losses.append(loss_value)

        pred_id = torch.argmax(last_output,1) #1D
        pred_id = pred_id.cpu().numpy() # (batch_size, vocab_len)　のはず
        val_ansid = val_ansid.cpu().numpy()
        val_acc.append(((pred_id == val_ansid).sum()/ len(val_ansid) ))
    
    take_checkpoint = (epoch != 0) and (epoch % check_freq == 0)
    increment_curriculum = (epoch != 0) and (epoch % args.curriculum_freq == 0)
    take_checkpoint=False#~
    increment_curriculum=False#~

    # detach memory from graph
    #~?mhx = { k : (v.detach() if isinstance(v, var) else v) for k, v in mhx.items() }

    #last_save_losses.append(loss_value)

    summarize = (epoch % summarize_freq == 0)
    if summarize:
      loss = np.mean(last_save_losses)
      #llprint("\n\tAvg. Logistic Loss: %.4f\n" % (loss))
      if np.isnan(loss):
        raise Exception('nan Loss')
      print("epoch:",epoch," loss:",loss)
      val_loss = np.mean(last_val_losses)
      print("    val_loss:",val_loss)
      mean_acc =np.mean(val_acc)
      print("    val_acc:",mean_acc)
      if mean_acc>val_bestacc:
        val_bestacc =mean_acc
        val_bestloss=val_loss
        bestepoch=epoch
        torch.save(rnn.state_dict(), save_path)
      last_save_losses = []
      last_val_losses =[]
      val_acc=[]

    '''しばらくデバッグはしない
    if summarize and rnn.debug:
      loss = np.mean(last_save_losses)
      last_save_losses = []
      if args.memory_type == 'dnc':
        viz.heatmap(
            v['memory'],
            opts=dict(
                xtickstep=10,
                ytickstep=2,
                title='Memory, t: ' + str(epoch) + ', loss: ' + str(loss),
                ylabel='layer * time',
                xlabel='mem_slot * mem_size'
            )
        )
      if args.memory_type == 'dnc':
        viz.heatmap(
            v['link_matrix'][-1].reshape(args.mem_slot, args.mem_slot),
            opts=dict(
                xtickstep=10,
                ytickstep=2,
                title='Link Matrix, t: ' + str(epoch) + ', loss: ' + str(loss),
                ylabel='mem_slot',
                xlabel='mem_slot'
            )
        )
        viz.heatmap(
            v['rev_link_matrix'][-1].reshape(args.mem_slot, -1),
            opts=dict(
                xtickstep=10,
                ytickstep=2,
                title='Reverse Link Matrix, t: ' + str(epoch) + ', loss: ' + str(loss),
                ylabel='mem_slot',
                xlabel='mem_slot'
            )
        )
      elif args.memory_type == 'sdnc' or args.memory_type == 'dnc':
        viz.heatmap(
            v['precedence'],
            opts=dict(
                xtickstep=10,
                ytickstep=2,
                title='Precedence, t: ' + str(epoch) + ', loss: ' + str(loss),
                ylabel='layer * time',
                xlabel='mem_slot'
            )
        )
      viz.heatmap(
          v['read_weights'],
          opts=dict(
              xtickstep=10,
              ytickstep=2,
              title='Read Weights, t: ' + str(epoch) + ', loss: ' + str(loss),
              ylabel='layer * time',
              xlabel='nr_read_heads * mem_slot'
          )
      )
      viz.heatmap(
          v['write_weights'],
          opts=dict(
              xtickstep=10,
              ytickstep=2,
              title='Write Weights, t: ' + str(epoch) + ', loss: ' + str(loss),
              ylabel='layer * time',
              xlabel='mem_slot'
          )
      )
      viz.heatmap(
          v['usage_vector'] if args.memory_type == 'dnc' else v['usage'],
          opts=dict(
              xtickstep=10,
              ytickstep=2,
              title='Usage Vector, t: ' + str(epoch) + ', loss: ' + str(loss),
              ylabel='layer * time',
              xlabel='mem_slot'
          )
      )
    '''
    if increment_curriculum:
      sequence_max_length = sequence_max_length + args.curriculum_increment
      print("Increasing max length to " + str(sequence_max_length))

    if take_checkpoint:
      llprint("\nSaving Checkpoint ... "),
      check_ptr = os.path.join(ckpts_dir, 'step_{}.pth'.format(epoch))
      cur_weights = rnn.state_dict()
      T.save(cur_weights, check_ptr)
      llprint("Done!\n")

  #Test
  print("=====TEST=====")
  last_save_losses=[]
  accuracy_rates =[]
  test_loss=0
  test_acc=0

  rnn.load_state_dict(torch.load(save_path))
  rnn.eval()

  with torch.no_grad():
    for i,batch in enumerate(test_iter):
      llprint("\nIteration %d/%d" % (i, iterations))
      # We test now the learned generalization using sequence_max_length examples
      batch_size=batch.batch_size
      test_x,test_y,ans_id =prep_data(batch,vocab_size)
      if val_x.size()[0]!=args.batch_size:continue
      if rnn.debug:
        output, (chx, mhx, rv), v = rnn(test_x, (None, mhx, None), reset_experience=True, pass_through_memory=True)
      else:
        output, (chx, mhx, rv) = rnn(test_x, (None, mhx, None), reset_experience=True, pass_through_memory=True)
      last_output =output[:,-1,:]

      optimizer.zero_grad()
      loss = criterion(last_output,ans_id)
      loss_value = loss.item()
      last_save_losses.append(loss_value)
      pred_id = torch.argmax(last_output,1) #1D
      pred_id = pred_id.cpu().numpy() # (batch_size, vocab_len)　のはず
      ans_id = ans_id.cpu().numpy()
      accuracy_rates.append(((pred_id == ans_id).sum()/ len(ans_id) ))

      """try:
        print("\nReal value: ", ' = ' + str(int(target_output[0])))
        print("Predicted:  ", ' = ' + str(int(output // 1)) + " [" + str(output) + "]")
      except Exception as e:
        pass"""

    test_loss = np.mean(last_save_losses)
    test_acc =np.mean(accuracy_rates)
    print(" loss:",test_loss)
    print("accuracy_rate:",test_acc)

  return {"task":task,"best_epoch":bestepoch,
  "best_val_loss":val_bestloss,"best_val_acc":val_bestacc,
  "test_loss":test_loss,"test_acc":test_acc
  }

In [16]:
state=[]
for task in range(1,21):
    state .append(main_dnc_babi(task))

print(pd.DataFrame(state))

==========task: 1
batch_size: 50
VOCAB LEN PLUS: 21

----------------------------------------
DNC(21, 256, nr_cells=256, read_heads=4, cell_size=64, gpu_id=0, independent_linears=True)
DNC(
  (lstm_layer_0): LSTM(277, 256, num_layers=2, batch_first=True)
  (rnn_layer_memory_shared): Memory(
    (read_keys_transform): Linear(in_features=256, out_features=256, bias=True)
    (read_strengths_transform): Linear(in_features=256, out_features=4, bias=True)
    (write_key_transform): Linear(in_features=256, out_features=64, bias=True)
    (write_strength_transform): Linear(in_features=256, out_features=1, bias=True)
    (erase_vector_transform): Linear(in_features=256, out_features=64, bias=True)
    (write_vector_transform): Linear(in_features=256, out_features=64, bias=True)
    (free_gates_transform): Linear(in_features=256, out_features=4, bias=True)
    (allocation_gate_transform): Linear(in_features=256, out_features=1, bias=True)
    (write_gate_transform): Linear(in_features=256, out_

RuntimeError: CUDA out of memory. Tried to allocate 14.00 MiB (GPU 0; 10.92 GiB total capacity; 6.50 GiB already allocated; 9.12 MiB free; 7.21 GiB reserved in total by PyTorch)

: 

detachのとことかまだよくわかってない  
lstmはモデルの外に隠れ状態を出していなかった。いや出力として出てくるんだけど、モデルの入力として渡さない。この場合detachをしなくて問題なかった  
こちらではモデルの出力として出したものを、次に入力に入れる。この場合計算グラフをさかのぼっていく？でも↑の場合だと遡る？それともライブラリの方で値引き継ぎとdetachをする？  
あとこっちの場合、chxとかを入力にしなくてもいいのか気になる